_FlashBacktest.py_ Quick Start User Guide
=======================

This tutorial shows some of the features of *FlashBacktest.py*, a Python framework for [backtesting](https://www.investopedia.com/terms/b/backtesting.asp) trading strategies.

__FlashBacktest.py_ is a small, lightweight, extremely fast and accurate backtesting frame that uses two backtesting frameworks: a lower frame and a higher fram , that uses the latest Python structures and routines (Python 3.10+, Pandas, NumPy). It has a very small and simple API that is easy to remember and configure quickly to achieve meaningful results. The library does not really support stock picking, arbitrage trading strategies, or multi-asset portfolio rebalancing; Instead, it works with one tradable asset at a time and is best suited for optimizing entry and exit signal strategies for trades, decisions based on technical indicator values, and is a versatile interactive trading visualization and statistics tool.


## Data

_You bring your own data._ Backtesting ingests _all kinds of 
[OHLC](https://en.wikipedia.org/wiki/Open-high-low-close_chart)
data_ (stocks, forex, futures, crypto, ...) as a
[pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/10min.html)
with columns `'Open'`, `'High'`, `'Low'`, `'Close'` and (optionally) `'Volume'`.
Such data is widely obtainable, e.g. with packages:
* [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/),
* [Quandl](https://www.quandl.com/tools/python),
* [findatapy](https://github.com/cuemacro/findatapy),
* [yFinance](https://github.com/ranaroussi/yfinance),
* [investpy](https://investpy.readthedocs.io/),
  etc.

Besides these columns, **your data frames can have additional columns which are accessible in your strategies in a similar manner**.

DataFrame should ideally be indexed with a _datetime index_ (convert it with [`pd.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html));
otherwise a simple range index will do.

In [1]:
# Example OHLC daily data for AGLDUSDT Inc.
from binance.client import Client

timeframHigh = "1h"
timeframLOW = "1m"
symbol = "AGLDUSDT"

client = Client('', '')

AGLDUSDTF_rameHIGHER = client.get_historical_klines(symbol, timeframHigh)
AGLDUSDT_FrameLOW = client.get_historical_klines(symbol, timeframLOW)


Loading BokehJS ...

,Open,High,Low,Close,Volume
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800
2013-03-01,797.80,807.14,796.15,806.19,2175400


## Strategy

Let's create our first strategy to backtest on these AGLDUSDT data, a simple [moving average (MA) cross-over strategy](https://en.wikipedia.org/wiki/Moving_average_crossover).

_FlashBacktest.py_ doesn't ship its own set of _technical analysis indicators_. Users favoring TA should probably refer to functions from proven indicator libraries, such as
[TA-Lib](https://github.com/mrjbq7/ta-lib) or
[Tulipy](https://tulipindicators.org),
but for this example, we can define a simple helper moving average function ourselves:

In [2]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

A new strategy needs to extend 
[`Strategy`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy)
class and override its two abstract methods:
[`init()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.init) and
[`next()`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.next).

Method `init()` is invoked before the strategy is run. Within it, one ideally precomputes in efficient, vectorized manner whatever indicators and signals the strategy depends on.

Method `next()` is then iteratively called by the
[`Backtest`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Backtest)
instance, once for each data point (data frame row), simulating the incremental availability of each new full candlestick bar.

[`self.data`](https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Strategy.data).

In `init()`, we declare and **compute indicators indirectly by wrapping them in 
The wrapper is passed a values (our `SMA` array) along with any arguments to call it


In `next()`, we simply check whether the faster moving average has crossed the slower moving average. If this happens and is higher, we open a long buy limit at the close and TP: 20% above the closing price, SL: 10% below the closing price, if this happens and below, we open a sell order.


In [3]:
from FlashBacktest import Strategy



class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.data.sma1
        self.sma2 = self.data.sma2


    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.limit = self.data.Close[-1]
            self.tp =  self.data.Close[-1] +  (self.data.Close[-1]*0.20)
            self.sl = self.data.Close[-1] - (self.data.Close[-1]*0.10)
            self.buy(limit=self.limit ,tp=self.tp,sl= self.sl )

        elif (self.sma1[-2] > self.sma2[-2] and    # Ugh!
              self.sma1[-1] < self.sma2[-1]):
            self.limit = self.data.Close[-1]
            self.tp =  self.data.Close[-1] -  (self.data.Close[-1]*0.20)
            self.sl = self.data.Close[-1] + (self.data.Close[-1]*0.10)
            self.sell()
 

In `init()`, the whole series of points was available, whereas **in `next()`, the length of `self.data` and all declared indicators is adjusted** on each `next()` call so that `array[-1]` (e.g. `self.data.Close[-1]` or `self.sma1[-1]`) always contains the most recent value, `array[-2]` the previous value, etc. (ordinary Python indexing of ascending-sorted 1D arrays).

**Note**: `self.data` and any indicators wrapped with . If you prefer pandas Series or DataFrame objects, use `Strategy.data.<column>.s` or `Strategy.data.df` accessors respectively.


## Backtesting

Let's see how our strategy performs on historical AGLDUSDT data. The
`Backtest`
instance is initialized with OHLC data and a strategy _class_ (see API reference for additional options), and we begin with 10,000 units of cash and set broker's fess to realistic 0.01% ,use 20% from cash for order .

In [5]:
from FlashBackTesting import FlashBackTesting

bt = FlashBackTesting(AGLDUSDTF_rameHIGHER,AGLDUSDT_FrameLOW,SmaCross,ratio_entry=20,fess=0.001)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                       94.27
Equity Final [$]                     68935.12
Equity Peak [$]                      68991.22
Return [%]                             589.35
Buy & Hold Return [%]                  703.46
Max. Drawdown [%]                      -33.08
Avg. Drawdown [%]                       -5.58
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   93
Win Rate [%]                            53.76
Best Trade [%]                          57.12
Worst Trade [%]                        -16.63
Avg. Trade [%]                           1.96
Max. Trade Duration         121 days 00:00:00
Avg. Trade Duration          32 days 00:00:00
Profit Factor                            2.13
Expectancy [%]                           6.91
SQN                               

The columns should be self-explanatory.

In [11]:
stats['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.

,Equity,DrawdownPct,DrawdownDuration
2004-08-19,10000.00,0.00,NaT
2004-08-20,10000.00,0.00,NaT
2004-08-23,10000.00,0.00,NaT
2004-08-24,10000.00,0.00,NaT
2004-08-25,10000.00,0.00,NaT
...,...,...,...
2013-02-25,103035.53,0.05,NaT
2013-02-26,102952.33,0.05,NaT
2013-02-27,104206.83,0.04,NaT
2013-02-28,104391.43,0.04,NaT


In [12]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,87,20,60,114.65,185.23,6140.56,0.62,2004-09-17,2004-11-12,56 days
1,-87,60,69,184.86,175.80,788.18,0.05,2004-11-12,2004-11-26,14 days
2,96,69,71,176.15,180.71,437.61,0.03,2004-11-26,2004-11-30,4 days
3,-96,71,75,180.35,179.13,116.98,0.01,2004-11-30,2004-12-06,6 days
4,97,75,82,179.49,177.99,-145.33,-0.01,2004-12-06,2004-12-15,9 days
...,...,...,...,...,...,...,...,...,...,...
147,-121,2056,2085,738.65,687.78,6155.24,0.07,2012-10-16,2012-11-29,44 days
148,139,2085,2111,689.16,735.54,6447.44,0.07,2012-11-29,2013-01-08,40 days
149,-139,2111,2113,734.07,742.83,-1217.79,-0.01,2013-01-08,2013-01-10,2 days
150,136,2113,2121,744.32,735.99,-1132.29,-0.01,2013-01-10,2013-01-23,13 days
